In [ ]:
from tqdm.notebook import trange, tqdm
from tba_types import Award, Event, WLTRecord
from tba_session import session, ROOT_URL
from collections import defaultdict

In [2]:
TEAM_PAGES = 22

In [3]:
ALL_TEAM_KEYS: list[str] = []

for i in trange(TEAM_PAGES, desc="Getting all teams", unit="page"):
    ALL_TEAM_KEYS.extend(
        session.get(f"{ROOT_URL}/teams/{i}/keys").json()
    )

Getting all teams:   0%|          | 0/22 [00:00<?, ?page/s]

In [ ]:
TEAM_EVENT_AWARDS: dict[str, dict[str, list[Award]]] = {}
WLT_MAP: dict[tuple[str, str], WLTRecord] = defaultdict(
    lambda: WLTRecord(wins=0, losses=0, ties=0)
)

EVENT_KEY_MAP: dict[str, str] = {}

EVENT_MAP: dict[str, Event] = {}


for team_key in tqdm(

    ALL_TEAM_KEYS, desc="Getting team events", unit="team", leave=False

):

    TEAM_EVENT_AWARDS[team_key] = {}
    team_events: list[Event] = session.get(f"{ROOT_URL}/team/{team_key}/events").json()
    for event in sorted(team_events, key=lambda event: event["start_date"]):
        EVENT_MAP[event["key"]] = event
        if (
            not 0 <= event["event_type"] <= 5
            or event["year"] == 2021
            or "cancelled" in event["name"].lower()
        ):
            continue
        EVENT_KEY_MAP[event["key"]] = (
            event["key"]
            if event["parent_event_key"] is None
            else event["parent_event_key"]
        )

        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[event["key"]]] = []

    team_awards: list[Award] = session.get(f"{ROOT_URL}/team/{team_key}/awards").json()
    for award in team_awards:
        if (
            award["event_key"] not in EVENT_KEY_MAP
            or award["award_type"] == 14
            or award["award_type"] == 39
            or award["award_type"] == 68
            # or not 3 <= award["award_type"] <= 5
        ):
            continue
        if EVENT_KEY_MAP[award["event_key"]] not in TEAM_EVENT_AWARDS[team_key]:
            TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[award["event_key"]]] = []
        TEAM_EVENT_AWARDS[team_key][EVENT_KEY_MAP[award["event_key"]]].append(award)

for event_key, map_to in tqdm(EVENT_KEY_MAP.items()):
    statuses: dict[str, dict] = session.get(f"{ROOT_URL}/event/{event_key}/teams/statuses").json()
    rankings: list[dict] = session.get(f"{ROOT_URL}/event/{event_key}/rankings").json()
    if rankings is None:
        continue
    qual_records: dict[str, WLTRecord] = dict(
        (team["team_key"], team["record"]) for team in rankings if "record" in team
    )
    for team_key, status in statuses.items():
        if status is None or "playoff" not in status or status["playoff"] is None or "record" not in status["playoff"]:
            continue
        quals_record = qual_records.get(team_key, None)
        playoffs_record: WLTRecord = status["playoff"]["record"]
        if not quals_record or not playoffs_record:
            continue
        record: WLTRecord = WLTRecord(
            wins=quals_record["wins"] + playoffs_record["wins"],
            losses=quals_record["losses"] + playoffs_record["losses"],
            ties=quals_record["ties"] + playoffs_record["ties"],
        )
        WLT_MAP[(team_key, map_to)]["wins"] += record["wins"]
        WLT_MAP[(team_key, map_to)]["losses"] += record["losses"]
        WLT_MAP[(team_key, map_to)]["ties"] += record["ties"]

Getting team events:   0%|          | 0/8831 [00:00<?, ?team/s]

  0%|          | 0/2332 [00:00<?, ?it/s]

{}
{'alliance': {'backup': None, 'name': 'Alliance 1', 'number': 1, 'pick': 0}, 'alliance_status_str': '<b>Captain</b> of <b>Alliance 1</b>', 'last_match_key': '2010wat_f1m2', 'next_match_key': None, 'overall_status_str': 'Team 1114 competed in the playoffs as the <b>Captain</b> of <b>Alliance 1</b> and <b>won the event</b> with a playoff record of <b>6-0-0</b>.', 'playoff': {'current_level_record': {'losses': 0, 'ties': 0, 'wins': 2}, 'level': 'f', 'playoff_average': None, 'playoff_type': None, 'record': {'losses': 0, 'ties': 0, 'wins': 6}, 'status': 'won'}, 'playoff_status_str': '<b>Won the event</b> with a playoff record of <b>6-0-0</b>', 'qual': {'num_teams': 30, 'ranking': {'dq': 0, 'matches_played': 9, 'qual_average': None, 'rank': 1, 'record': None, 'sort_orders': [253.0, 96.0, 12.0], 'team_key': 'frc1114'}, 'sort_order_info': [{'name': 'Seeding Score', 'precision': 0}, {'name': 'Coopertition Bonus', 'precision': 0}, {'name': 'Hanging Points', 'precision': 0}], 'status': 'comple

In [40]:
PERFECT_EVENTS: list[tuple[str, str, WLTRecord, list[Award]]] = []

for team_key, events in tqdm(
    TEAM_EVENT_AWARDS.items(),
    desc="Finding perfect events",
    unit="team",
):
    for event_key, awards in events.items():
        if len(awards) == 0:
            continue
        record = WLT_MAP[(team_key, event_key)]
        if record["wins"] != 0 and record["losses"] == 0 and record["ties"] == 0:
            if (
                len(awards) >= 3
                and any(award["award_type"] == 0 for award in awards)
                and any(award["award_type"] == 1 for award in awards)
            ):
                PERFECT_EVENTS.append(
                    (
                        team_key,
                        event_key,
                        record,
                        awards,
                    )
                )

Finding perfect events:   0%|          | 0/8831 [00:00<?, ?team/s]

In [41]:
WLT_MAP[("frc1114", "2010wat")]

{'wins': 0, 'losses': 0, 'ties': 0}

In [42]:
for team_key, event_key, record, awards in sorted(
    PERFECT_EVENTS,
    key=lambda x: int(x[0][3:]),
    reverse=True,
):
    print(
        f"### [{team_key[3:]}](https://thebluealliance.com/team/{team_key[3:]}) at [{EVENT_MAP[event_key]['year']} {EVENT_MAP[event_key]['name']}](https://thebluealliance.com/event/{event_key}): {record['wins']}-{record['losses']}-{record['ties']} with {len(awards)} awards:"
    )
    for award in awards:
        print(f" - {award['name']}")